In [1]:
#importing necessary libraries
from __future__ import print_function
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
#from imageai.Detection import VideoObjectDetection
import os
import sys
from random import randint
from math import ceil, sqrt
import natsort
import pandas as pd
import random
from keras.applications.vgg16 import VGG16
import pickle
from DataLoader import  DataGenerator

Matplotlib is building the font cache using fc-list. This may take a moment.
Using TensorFlow backend.



In [4]:
user = 'aws'

if user == 'siddhi':
    path_videos = 'C:/RoadCrossingAssistant/Data/Videos/'
    path_labels_csv = 'C:/RoadCrossingAssistant/Data/labels_framewise_csv.csv'
    path_labels_list = 'C:/RoadCrossingAssistant/Data/labels_framewise_list.pkl'
    path_frames = 'C:/RoadCrossingAssistant/Data/Frames/'

elif user == 'yagnesh':
    path_videos = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/videos/'
    path_labels_csv = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.csv'
    path_labels_list = '/home/yagnesh/Study/Machine Learning/ML projects/RoadCrossingAssistant_Arrays/labels_framewise.pkl'

elif user == 'aws':
    path_videos = '/home/ubuntu/Data/Videos/'
    path_labels_csv = '/home/ubuntu/Data/labels_framewise_csv.csv'
    path_labels_list = '/home/ubuntu/Data/labels_framewise_list.pkl'
    path_frames = '/home/ubuntu/Data/Frames/'


# frame-wise labels array
open_file = open(path_labels_list, "rb")
labels_list = pickle.load(open_file)
open_file.close()

In [5]:
#Perform train-test-validation split(62-24-18)

x = np.arange(1, 105)
np.random.shuffle(x)
#np.random.seed(42)
videos_validation = x[:18]
videos_test = x[18: 18+24]
videos_train = x[18+24: ]

# videos_train = [videos[ind] for ind in indices_train]
# videos_test = [videos[ind] for ind in indices_test]

# labels_train_loaded = [labels_list[ind] for ind in indices_train]
# labels_test_loaded = [labels_list[ind] for ind in indices_test]

# print('len of videos_train: ', len(videos_train))
# print('len of videos_test: ', len(videos_test))
# print('len of labels_train_loaded: ', len(labels_train_loaded))
# print('len of labels_test_loaded: ', len(labels_test_loaded))

print(videos_train, len(videos_train))
print(videos_test, len(videos_test))
print(videos_validation, len(videos_validation))


[  8  65  10  41  61   6  28  80  59  37  46  55  49   5  51  18  62  27
  98  16  88  75  35  82  69  86   9  85   7  67  30  33  25  74  19  97
  44  73  72  40  36 103  52  12 100  48  14  56  13  17  22  91  99  83
  32  96  53  20  76  45  42  11] 62
[ 29  31  39   3  70 101  89  92  90  66  64  63  78  58  81  87  38   2
  94  50  26  21   4  15] 24
[ 77  84  93  57  71  95  23  43 102  68  34  60 104  54   1  47  24  79] 18


In [7]:
list_IDs_train = {}
list_IDs_test = {}
list_IDs_validation = {}

for vid in videos_train:

    video = path_videos + "video" + str(vid) + ".MOV"
    #print("starting " + video)
    cap = cv2.VideoCapture(video)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    cv2.destroyAllWindows() 
    list_IDs_train[vid] = no_frames

for vid in videos_test:

    video = path_videos + "video" + str(vid) + ".MOV"
    #print("starting " + video)
    cap = cv2.VideoCapture(video)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    cv2.destroyAllWindows() 
    list_IDs_test[vid] = no_frames

for vid in videos_validation:

    video = path_videos + "video" + str(vid) + ".MOV"
    #print("starting " + video)
    cap = cv2.VideoCapture(video)
    no_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    cap.release()
    cv2.destroyAllWindows() 
    list_IDs_validation[vid] = no_frames

print(list_IDs_train)
print(len(list_IDs_train))

print(list_IDs_test)
print(len(list_IDs_test))

print(list_IDs_validation)
print(len(list_IDs_validation))

{8: 150, 65: 180, 10: 150, 41: 180, 61: 180, 6: 210, 28: 180, 80: 390, 59: 180, 37: 180, 46: 180, 55: 180, 49: 180, 5: 240, 51: 180, 18: 210, 62: 210, 27: 180, 98: 300, 16: 210, 88: 510, 75: 180, 35: 210, 82: 480, 69: 180, 86: 330, 9: 150, 85: 780, 7: 210, 67: 180, 30: 180, 33: 180, 25: 180, 74: 180, 19: 180, 97: 390, 44: 180, 73: 180, 72: 180, 40: 210, 36: 180, 103: 330, 52: 180, 12: 180, 100: 450, 48: 180, 14: 180, 56: 180, 13: 240, 17: 330, 22: 150, 91: 690, 99: 390, 83: 480, 32: 150, 96: 420, 53: 180, 20: 210, 76: 210, 45: 180, 42: 180, 11: 210}
62
{29: 180, 31: 210, 39: 180, 3: 180, 70: 180, 101: 390, 89: 450, 92: 420, 90: 570, 66: 180, 64: 180, 63: 180, 78: 480, 58: 180, 81: 570, 87: 570, 38: 180, 2: 150, 94: 330, 50: 180, 26: 210, 21: 180, 4: 150, 15: 180}
24
{77: 300, 84: 750, 93: 240, 57: 180, 71: 180, 95: 420, 23: 180, 43: 240, 102: 240, 68: 180, 34: 150, 60: 180, 104: 240, 54: 180, 1: 210, 47: 180, 24: 240, 79: 390}
18


In [8]:
# Generators
training_generator = DataGenerator(list_IDs = list_IDs_train, folder_path = path_frames)
validation_generator = DataGenerator(list_IDs = list_IDs_validation, folder_path = path_frames)
testing_generator = DataGenerator(list_IDs = list_IDs_test, folder_path = path_frames)


In [14]:
from tensorflow.keras.layers import Flatten, Dense
import keras
import tensorflow
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras import layers 
from tensorflow.keras.applications import MobileNet

base_model = MobileNet(input_shape = (360, 640, 3), include_top = False, weights = 'imagenet')

for layer in base_model.layers:
    layer.trainable = False

x = layers.Flatten()(base_model.output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.Dense(2, activation='softmax')(x)

model = tensorflow.keras.models.Model(base_model.input, x)
model.compile(
    loss='binary_crossentropy',
    optimizer=tensorflow.keras.optimizers.Adam(),
    metrics=['accuracy'])

model.summary()

C:\Anaconda\envs\roadcross\lib\site-packages\keras_applications\mobilenet.py:207: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  warnings.warn('`input_shape` is undefined or non-square, '
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 360, 640, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 361, 641, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 180, 320, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 180, 320, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)

In [7]:
#history = model.fit_generator(train_generator, shuffle='true', epochs=1, verbose=1, batch_size=16)

history = model.fit_generator(generator=training_generator, validation_data=validation_generator, epochs=15, verbose=1)

[0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 1 1 0 1 1 0 1 0 0 0 0 0 1 0 0 1 0 1 1 1 0 1 0 0 0 1 0]
[0 0 1 1 0 0 0 0 0 0 1 0 1 0 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 0 0 0]
[1 0 0 0 0 0 1 0 0 1 0 1 1 1 1 1 0 0 1 1 0 0 0 0 1 0 0 0 1 0 1 1]
[0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0]
[1 1 1 0 1 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 0 0 0 1 0 0 1]
[0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 1 1 1 1]
[1 1 1 1 1 0 1 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 1 0 0 0 0 0 1 0 1]
[0 0 1 0 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 1 1 0 0 0 1 1 1 1 1 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1]
  2/666 [..............................] - ETA: 8:10:52 - loss: 1.4485 - acc: 0.6406 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 

KeyboardInterrupt: 

In [1]:
# import random
# from PIL import Image 

# dim = (1080, 1920)
# X = np.empty((4, *dim, 3))
# y = np.empty((4), dtype=int)

# video_ids = [2,3]

# c = 0

# for vid in video_ids:

#     frame_ids = random.sample(list(range(list_IDs_train[vid])), 2) # generate "no_frames_per_video" random frames
#             #max value of frame given by self.list_ids[vid]
#     labels_temp = np.load(path_frames + "video" + str(vid) + "/labels" + str(vid) + ".npy")

#     for fid in frame_ids:
#         im = Image.open(path_frames + "video" + str(vid) + "/frame" + str(fid) + ".jpg")

#         #X[c, ] = np. asarray(im)

#         d = np.asarray(im)
#         frame_resized = cv2.resize(d, (640, 360), interpolation = cv2.INTER_AREA)

#         print(d.shape, frame_resized.shape)
                
#         y[c] = labels_temp[fid]

#         c = c + 1

# print(c)

# print(X.shape, y.shape)

# print(y)

# print(keras.utils.to_categorical(y, num_classes=2))

In [67]:
print("Evaluate on test data")
results = model.evaluate(testing_generator)
print("test loss, test acc:", results)

print("Evaluate on train data")
results = model.evaluate(training_generator)
print("train loss, trai acc:", results)

# Generate predictions (probabilities -- the output of the last layer)
# on new data using `predict`
# print("Generate predictions")
# predictions = model.predict(frames_test)
# print("predictions shape:", predictions.shape)
# print(predictions[:10])




Evaluate on test data
567/567 [==============================] - 83s 147ms/sample - loss: 8.3812 - acc: 0.4550
test loss, test acc: [8.381198916695947, 0.45502645]
Generate predictions
predictions shape: (567, 2)


In [2]:
# from sklearn.metrics import average_precision_score
# average_precision = average_precision_score(np.argmax(y_test, axis = 1), np.argmax(predictions, axis = 1))

# print('Average precision-recall score: {0:0.2f}'.format(
#       average_precision))